## Python for Finance 8 - Modern Portfolio Theory

In [ ]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

#Statsmodels is a great library we can use to run regressions.
import statsmodels.api as sm
# Seaborn extends the capabilities of Matplotlib
import seaborn as sns
# Used for calculating regressions
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

#### Dates & Other Constants

In [ ]:
# Define path to files
PATH = "../data/stock-list/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 1
S_DATE_STR = f"{ST_YEAR}-{ST_MONTH}-{ST_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
EN_YEAR = 2021
EN_MONTH = 12
EN_DAY = 31
EN_DATE_STR = f"{EN_YEAR}-{EN_MONTH}-{EN_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

risk_free_rate = 0.0125 # Approximate 10 year bond rate

#### Get Stock File Names in a List

In [ ]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

tickers.sort()
len(tickers)

#### Function that Returns a Dataframe from a CSV

In [ ]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col='Date', 
                         parse_dates=True)
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

#### Merge Multiple Stocks in One Dataframe by Column Name

In [ ]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

#### Markowitz Portfolio Optimization

Harry Markowitz proved that you could make what is called an efficient portfolio. That is a portfolio that optimizes return while also minimizing risk. We don't benefit from analyzing individual securities at the same rate as if we instead considered a portfolio of stocks.

We do this by creating portfolios with stocks that are not correlated. We want to calculate expected returns by analyzing the returns of each stock multiplied by its weight.

$w_1r_1 + w_2r_2 = r_p$

The standard deviation of the portfolio is found this way. Sum multiple calculations starting by finding the product of the first securities weight squared times its standard deviation squared. The middle is 2 times the correlation coefficient between the stocks. And, finally add those to the weight squared times the standard deviation squared for the second security.

$(w_1 \sigma_1 + w_2 \sigma_2)^2 = w_1^2 \sigma_1^2 + 2w_1 \sigma_1w_2 \sigma_2 \rho_1 + w_2^2 \sigma_2^2$

#### Plotting an Efficient Frontier

#### Stock Portfolio

In [ ]:
# Here I added the top 30 for each sector. You can obviously add as many as 
# you like
# Then to keep the portfolio smaller I eliminated stocks that had an 
# allocation less than 1% to get this final list
# I showed in part 4 of this series how to get the top sector stocks
port_list = ["GNRC", "CPRT", "ODFL", 
            "AMD", "PAYC",
            "CHTR",
            "MKC", "PG",
            "PGR",
            "NEM",
            "CCI",
            "COG"]
num_stocks = len(port_list)
num_stocks

#### Merge all Stock Price Data

In [ ]:
mult_df = merge_df_by_column_name('Adj Close',  '2018-01-01', 
                                  '2021-09-01', *port_list)
mult_df

#### Plot Growth of Investment over Total Dataset

In [ ]:
(mult_df / mult_df.iloc[0] * 100).plot(figsize=(16, 9))

#### Mean Returns

In [ ]:
returns = np.log(mult_df / mult_df.shift(1))
mean_ret = returns.mean() * 252 # 252 average trading days per year
mean_ret

#### Covariance

In [ ]:
returns.cov() * 252

#### Correlation

In [ ]:
returns.corr()

#### Generate Random Weight Equal to 1

In [ ]:
# Generate 11 random values that sum to 1
weights = np.random.random(11)
weights /= np.sum(weights)
print('Weights :', weights)
print('Total Weight :', np.sum(weights))

#### Calculating Return of Portfolio

In [ ]:
# Provide return of portfolio using random weights over the whole dataset
np.sum(weights * returns.mean()) * 252

#### Calculating Volatility

In [ ]:
np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))

#### Analyzing Returns & Risk of 10000 Combinations

In [ ]:
p_ret = [] # Returns list
p_vol = [] # Volatility list
p_SR = [] # Sharpe Ratio list
p_wt = [] # Stock weights list


for x in range(10000):
    # Generate random weights
    p_weights = np.random.random(num_stocks)
    p_weights /= np.sum(p_weights)
    
    # Add return using those weights to list
    ret_1 = np.sum(p_weights * returns.mean()) * 252
    p_ret.append(ret_1)
    
    # Add volatility or standard deviation to list
    vol_1 = np.sqrt(np.dot(p_weights.T, np.dot(returns.cov() * 252, p_weights)))
    p_vol.append(vol_1)
    
    # Get Sharpe ratio
    SR_1 = (ret_1 - risk_free_rate) / vol_1
    p_SR.append(SR_1)
    
    # Store the weights for each portfolio
    p_wt.append(p_weights)
    
# Convert to Numpy arrays
p_ret = np.array(p_ret)
p_vol = np.array(p_vol)
p_SR = np.array(p_SR)
p_wt = np.array(p_wt)

p_ret, p_vol, p_SR, p_wt

#### Plotting the Efficient Frontier

In [ ]:
# Create a dataframe with returns and volatility
ports = pd.DataFrame({'Return': p_ret, 'Volatility': p_vol})

ports.plot(x='Volatility', y='Return', kind='scatter', figsize=(16, 9))

#### Sharpe Ratio

People want to maximize returns while avoiding as much risk as possible. William Sharpe created the Sharpe Ratio to find the portfolio that provides the best return for the lowest amount of risk.

*Sharpe Ratio* = $ \frac{r_i - r_f}{\sigma_i}$

$r_f = $ Risk Free Rate

$r_i = $ Rate of Return of the stock

$\sigma_i = $ Standard Deviation of the Stock

As return increases so does the Sharpe Ratio, but as Standard Deviation increase the Sharpe Ratio decreases.

In [ ]:
# Return the index of the largest Sharpe Ratio
SR_idx = np.argmax(p_SR)

# Find the ideal portfolio weighting at that index
i = 0
while i < num_stocks:
    print("Stock : %s : %2.2f" % (port_list[i], (p_wt[SR_idx][i] * 100)))
    i += 1
    
# Find volatility of that portfolio
print("\nVolatility :", p_vol[SR_idx])
      
# Find return of that portfolio
print("Return :", p_ret[SR_idx])

#### Get Number of Shares & Cost Based on Weights

In [ ]:
# I'm going to use a similar portfolio that has slightly easier percents to work with

# Calculated Weighting
# Stock : GNRC : 21.90 
# Stock : CPRT : 15.63 
# Stock : ODFL : 23.83
# Stock : AMD : 11.09
# Stock : PAYC : 1.86
# Stock : CHTR : 3.55
# Stock : MKC : 2.87
# Stock : PG : 1.09
# Stock : PGR : 2.86
# Stock : NEM : 10.11
# Stock : CCI : 4.34
# Stock : COG : 0.87

Find percentage closest to 1 and use it as the basis for calculating all other shares Since PG is 1% : GNRC is 21.90 : Multiply cost of PG (118.20) by 22 (21.9 Rounded up) and then divide that by the cost of GNRC to get the shares to buy for GNRC

118.20 22 = 2,600.4 / 102.01 = 25.49 (25 Shares) 102.01 =

In situations in which prices don't work either discard stock or force 1 share

In [ ]:
def get_port_shares(one_price, force_one, wts, prices):
    # Gets number of stocks to analyze
    num_stocks = len(wts)
    
    # Holds the number of shares for each
    shares = []
    
    # Holds Cost of shares for each
    cost_shares = []
    
    i = 0
    while i < num_stocks:
        # Get max amount to spend on stock 
        max_price = one_price * wts[i]
        
        # Gets number of shares to buy and adds them to list
        num_shares = int(max_price / prices[i])
        
        # If the user wants to force buying one share do it
        if(force_one & (num_shares == 0)):
            num_shares = 1
        
        shares.append(num_shares)
        
        # Gets cost of those shares and appends to list
        cost = num_shares * prices[i]
        cost_shares.append(cost)
        i += 1
        
    return shares, cost_shares

#### Get Portfolio Weighting

In [ ]:
def get_port_weighting(share_cost):
    
    # Holds weights for stocks
    stock_wts = []
    # All values summed
    tot_val = sum(share_cost)
    print("Total Investment :", tot_val)
    
    for x in share_cost:
        stock_wts.append(x / tot_val)
    return stock_wts

#### Returns the Value of Portfolio by Date

In [ ]:
def get_port_val_by_date(date, shares, tickers):
    port_prices = merge_df_by_column_name('Adj Close',  date, 
                                  date, *port_list)
    # Convert from dataframe to Python list
    port_prices = port_df_start.values.tolist()
    # Trick that converts a list of lists into a single list
    port_prices = sum(port_prices, [])
    return port_prices

#### Approximate Portfolio Weighting Based on Request

In [ ]:
port_list = ["GNRC", "CPRT", "ODFL", "AMD", "PAYC", "CHTR", "MKC", 
             "PG", "PGR", "NEM", "CCI", "COG"]

port_wts = [22, 16, 24, 11, 2, 4, 3, 1, 3, 10, 4, 1]

# Get all stock prices on the starting date
port_df_start = merge_df_by_column_name('Adj Close',  '2020-01-02', 
                                  '2020-01-02', *port_list)
# Convert from dataframe to Python list
port_prices = port_df_start.values.tolist()
# Trick that converts a list of lists into a single list
port_prices = sum(port_prices, [])

# Calculated Weighting
# Stock : GNRC : 21.90 
# Stock : CPRT : 15.63 
# Stock : ODFL : 23.83
# Stock : AMD : 11.09
# Stock : PAYC : 1.86
# Stock : CHTR : 3.55
# Stock : MKC : 2.87
# Stock : PG : 1.09
# Stock : PGR : 2.86
# Stock : NEM : 10.11
# Stock : CCI : 4.34
# Stock : COG : 0.87

tot_shares, share_cost = get_port_shares(118.20, True, port_wts, port_prices)
print("Shares :", tot_shares)
print("Share Cost :", share_cost)

# Get list of weights for stocks
stock_wts = get_port_weighting(share_cost)
print("Stock Weights :", stock_wts)

# Get value at end of year
get_port_val_by_date('2020-12-31', tot_shares, port_list)